In [5]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import pickle
from keras.utils.np_utils import to_categorical
import keras_tuner as kt
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [6]:
#important landmarks
landmarks = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]



HEADERS = ["label"]

for lm in landmarks:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [7]:
print(HEADERS)

['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']


In [4]:
def describe_dataset(dataset_path: str):


    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [5]:
# load dataset
df = describe_dataset("/content/drive/MyDrive/train_trest_bc/bc/train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 15372 
Number of columns: 37

Labels: 
C    8238
L    7134
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [8]:
with open("/content/drive/MyDrive/train_trest_bc/model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [11]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [13]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")

In [27]:
def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [28]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)


In [24]:
def model_builder(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_layer_6 = hp.Int('layer_6', min_value=32, max_value=512, step=32)
    hp_layer_7 = hp.Int('layer_7', min_value=32, max_value=512, step=32)
    hp_layer_8 = hp.Int('layer_8', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_3, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_4, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_5, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_6, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_7, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_8, activation=hp_activation))
    model.add(tf.keras.layers.Dense(2, activation = "softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [30]:
tuner_dl = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_dl.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

In [2]:
model_dl = get_best_model(tuner_dl)
history=model_dl.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

NameError: name 'get_best_model' is not defined

In [32]:
describe_model(model_dl)

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd5c3197a30>
Layer-2: 320 units, func:  <function relu at 0x7fd5c3197a30>
Layer-3: 320 units, func:  <function relu at 0x7fd5c3197a30>
Layer-4: 512 units, func:  <function relu at 0x7fd5c3197a30>
Layer-5: 256 units, func:  <function relu at 0x7fd5c3197a30>
Layer-6: 416 units, func:  <function relu at 0x7fd5c3197a30>
Layer-7: 64 units, func:  <function relu at 0x7fd5c3197a30>
Layer-8: 256 units, func:  <function relu at 0x7fd5c3197a30>
Layer-9: 128 units, func:  <function relu at 0x7fd5c3197a30>
Layer-10: 2 units, func:  <function softmax at 0x7fd5c3197010>


In [1]:
epoch = range(len(history.history["loss"]))
plt.figure()
plt.plot(epoch, history.history['accuracy'], 'red', label = 'train_accuracy')
plt.plot(epoch, history.history['val_accuracy'], 'blue', label = 'val_accuaracy')
plt.title("Training Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("accuracy")
plt.legend()

NameError: name 'history' is not defined

In [ ]:
epoch = range(len(history.history["loss"]))
plt.figure()
plt.plot(epoch, history.history['loss'], 'red', label = 'train_loss')
plt.plot(epoch, history.history['val_loss'], 'blue', label = 'val_loss')
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()

In [38]:


# Evaluate model
predict_x = model_dl.predict(x_test, verbose=False) 
y_pred_class = np.argmax(predict_x, axis=1)
y_test_class = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
(p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])

train_set_results = [( 'Model', round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm )]

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
df = pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

df

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,Model,"[1.0, 0.994]","[0.995, 1.0]","[0.997, 0.997]","[[1670, 9], [0, 1396]]"


In [39]:
model_dl.save("/content/drive/MyDrive/train_trest_bc/model/bicep_dl.h5")